### Importing libraries

In [ ]:
import numpy as np 
import pandas as pd 
import os
import re
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from kaggle_datasets import KaggleDatasets
import tensorflow as tf

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
# Let's use 192x192 by default
image_size = (192, 192)

# Data dirs
data_gcs = KaggleDatasets().get_gcs_path('tpu-getting-started')

# Subdirs by image size
data_dir_by_size = {
    (512, 512): '/tfrecords-jpeg-512x512',
    (331, 331): '/tfrecords-jpeg-331x331',
    (224, 224): '/tfrecords-jpeg-224x224',
    (192, 192): '/tfrecords-jpeg-192x192'
}
subdir = data_dir_by_size[image_size]
# Paths to data files
train_file_names = tf.io.gfile.glob(data_gcs + subdir + '/train' + '/*.tfrec')
val_file_names = tf.io.gfile.glob(data_gcs + subdir + '/val' + '/*.tfrec')
test_file_names = tf.io.gfile.glob(data_gcs + subdir + '/test' + '/*.tfrec')

In [ ]:
# Classification Categories in our Dataset
flower_categories = [
    'pink primrose',     'hard-leaved pocket orchid', 'canterbury bells',
    'sweet pea',         'wild geranium',             'tiger lily',
    'moon orchid',       'bird of paradise',          'monkshood',
    'globe thistle',     'snapdragon',                'colt\'s foot',
    'king protea',       'spear thistle',             'yellow iris',
    'globe-flower',      'purple coneflower',         'peruvian lily',
    'balloon flower',    'giant white arum lily',     'fire lily',
    'pincushion flower', 'fritillary',                'red ginger',
    'grape hyacinth',    'corn poppy',                'prince of wales feathers',
     'stemless gentian',  'artichoke',                 'sweet william',
    'carnation',         'garden phlox',              'love in the mist',
    'cosmos',            'alpine sea holly',          'ruby-lipped cattleya',
    'cape flower',       'great masterwort',          'siam tulip',
    'lenten rose',       'barberton daisy',           'daffodil',
    'sword lily',        'poinsettia',                'bolero deep blue',
    'wallflower',        'marigold',                  'buttercup',
    'daisy',             'common dandelion',          'petunia',
    'wild pansy',        'primula',                   'sunflower',
    'lilac hibiscus',    'bishop of llandaff',        'gaura',
    'geranium',          'orange dahlia',             'pink-yellow dahlia',
    'cautleya spicata',  'japanese anemone',          'black-eyed susan',
    'silverbush',        'californian poppy',         'osteospermum',
    'spring crocus',     'iris',                      'windflower',
    'tree poppy',        'gazania',                   'azalea',
    'water lily',        'rose',                      'thorn apple',
    'morning glory',     'passion flower',            'lotus',
    'toad lily',         'anthurium',                 'frangipani',
    'clematis',          'hibiscus',                  'columbine',
    'desert-rose',       'tree mallow',               'magnolia',
    'cyclamen ',         'watercress',                'canna lily',
    'hippeastrum ',      'bee balm',                  'pink quill',
    'foxglove',          'bougainvillea',             'camellia',
    'mallow',            'mexican petunia',           'bromelia',
    'blanket flower',    'trumpet creeper',           'blackberry lily',
    'common tulip',      'wild rose'
]

print('Number of flower categories:', len(flower_categories))

In [ ]:
def decode_image(image_data):
    """Decodes JPEG data and return a normalized image.
    
    WARNING: you may need a different normalization if you
    use VGG-like networks.
    """
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.reshape(image, [*image_size, 3])
    return image

def read_labeled_tfrecord(example):
    """
    Converts a single record in labeled dataset (i.e. train and validation
    sets) to the more convenient format (image, label)
    """
    example = tf.io.parse_single_example(
        serialized=example,
        features={
            'image': tf.io.FixedLenFeature([], tf.string),
            'class': tf.io.FixedLenFeature([], tf.int64),
        }
    )
    
    image = decode_image(example['image'])
    label = tf.cast(example['class'], tf.int32)
    return image, label

def read_unlabeled_tfrecord(example):
    """
    Converts a single record in labeled dataset (i.e. test
    set) to the more convenient format (image, id)
    """
    example = tf.io.parse_single_example(
        serialized=example,
        features={
            'image': tf.io.FixedLenFeature([], tf.string),
            'id': tf.io.FixedLenFeature([], tf.string),
        }
    )
    
    image = decode_image(example['image'])
    idnum = example['id']
    return image, idnum

def load_dataset(filenames, labeled=True, ordered=False):
    """
    Given a list of `*.tfrec` file names, converts them into a `tf.data.Dataset`
    object that yields elements of format (image, label) or (image, id)
    
    # Arguments
        filenames: list of paths to `*.tfrec` files
        labeled: if True, the resulting dataset will yield data in format
            (image, label). Otherwise it will yield in format (image, id)
        ordered: whether to shuffle the dataset (not desirable for test/val)
        
    # Returns
        a `tf.data.Dataset` object that holds memory map to `*.tfrec` files
    """
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = ordered
    
    dataset = tf.data.TFRecordDataset(
        filenames, num_parallel_reads=tf.data.experimental.AUTOTUNE)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(
        read_labeled_tfrecord if labeled else read_unlabeled_tfrecord,
        num_parallel_calls=tf.data.experimental.AUTOTUNE)
    return dataset



In [ ]:
def data_augment(image, label):
    image = tf.image.random_flip_left_right(image)
    return image, label 

def get_training_dataset():
    dataset = load_dataset(train_file_names, labeled=True)
    dataset = dataset.map(data_augment, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.repeat() # the training dataset must repeat for several epochs
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def get_validation_dataset(ordered=False):
    dataset = load_dataset(val_file_names, labeled=True, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache()
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

def get_test_dataset(ordered=False):
    dataset = load_dataset(test_file_names, labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    return dataset


def count_data_items(filenames):
    """
    There's no way to obtain explicitly the number of elements in each dataset
    (see: https://stackoverflow.com/questions/40472139/), but we can infer that
    from file names, i.e. flowers00-230.tfrec = 230 data items
    """
    return np.sum([
        int(re.compile(r"-([0-9]*)\.").search(filename).group(1))
        for filename in filenames])


# Import Datasets
train_dataset = load_dataset(train_file_names, labeled=True, ordered=False)
val_dataset = load_dataset(val_file_names, labeled=True, ordered=False)
test_dataset = load_dataset(test_file_names, labeled=False, ordered=True)

# Calculate number of items
num_training_samples = count_data_items(train_file_names)
num_validation_samples = count_data_items(val_file_names)
num_testing_samples = count_data_items(test_file_names)

In [ ]:
print('Train samples:', num_training_samples, end=', ')
print('Val samples:', num_validation_samples, end=', ')
print('Test samples:', num_testing_samples)

In [ ]:
BATCH_SIZE = 128

train = get_training_dataset()
valid = get_validation_dataset()
test = get_test_dataset()

print("Training:", train)
print ("Validation:", valid)
print("Test:", test)

In [ ]:
from keras.models import model_from_json
# load json and create model
json_file = open('../input/models/model_IResNet2.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model_IResNet2 = model_from_json(loaded_model_json)
# load weights into new model
model_IResNet2.load_weights("../input/models/model_IResNet2.h5")
print("Loaded model from disk")

In [ ]:
# load json and create model
json_file = open('../input/models/model_dense.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model_dense = model_from_json(loaded_model_json)
# load weights into new model
model_dense.load_weights("../input/models/model_dense.h5")
print("Loaded model from disk")

In [ ]:
model_IResNet2.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy',metrics=['accuracy'])
model_dense.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy',metrics=['accuracy'])


In [ ]:
c = model_IResNet2.fit(train, validation_data = valid, epochs = 3, batch_size=128,steps_per_epoch = num_training_samples // 128)

In [ ]:
d = model_dense.fit(train, validation_data = valid, epochs = 3, batch_size=128,steps_per_epoch = num_training_samples // 128)

In [ ]:
test_ds = get_test_dataset(ordered=True) 
test_images_ds = test_ds.map(lambda image, idnum: image)
pred1 = model_dense.predict(test_images_ds)
pred2 = model_IResNet2.predict(test_images_ds)

In [ ]:
probabilities = (pred1+ pred2)/2
predictions = np.argmax(probabilities, axis=-1)

In [ ]:
test_ids_ds = test_ds.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(num_testing_samples))).numpy().astype('U')
submission = pd.DataFrame({'id':test_ids,
                          'label':predictions})

In [ ]:
submission.to_csv('submission.csv',index=False)

#### If you liked the Notebook Please Upvote It !
#### Thank You